import libraries

In [1]:
import pyodbc
import pandas as pd
import numpy as np
import joblib
import time

preprocess data

In [2]:
def calculate_rms(row):
    return np.sqrt(np.mean(row**2))


def preprocess_csv(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    df = df.drop_duplicates()

    # Drop the "water level" column
    df.drop(df.columns[25], axis=1, inplace=True)

    # Check for null or 0 values
    null_or_zero_rows = df.isnull().any(axis=1) | (df == 0).any(axis=1)

    # Remove rows with null or 0 values
    df = df[~null_or_zero_rows]

    # Add header at the first row by deleting the existing row
    header = [
        "pump_Id",
        "date",
        "timestamp",
        "Vr",
        "Vy",
        "Vb",
        "Ir",
        "Iy",
        "Ib",
        "freq",
        "power",
        "pf",
        "MNDE_temp",
        "MDE_temp",
        "PNDE_temp",
        "PDE_temp",
        "MNDE_vib",
        "MDE_vib",
        "PNDE_vib",
        "PDE_vib",
        "speed",
        "pump_eff",
        "motor_eff",
        "combined_eff",
        "water_flow",
        "head",
    ]
    df.columns = header

    df["timestamp"] = pd.to_datetime(df["date"] + " " + df["timestamp"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.sort_values(by="timestamp")
    df['voltage_RMS'] = df[['Vr', 'Vy', 'Vb']].apply(calculate_rms,axis=1)
    df['current_RMS'] = df[['Ir', 'Iy', 'Ib']].apply(calculate_rms,axis=1)
    df["power"] = (np.sqrt(3) * df["voltage_RMS"] * df["current_RMS"] * df["pf"]) / 1000

    # Overwrite the original file with preprocessed data
    df.to_csv("dataset/preprocess.csv", index=False)

preprocess_csv("dataset/combined_data.csv")

load classifier models

In [3]:
loaded_model_V = joblib.load("models/voltage_classifier.joblib")
loaded_model_I = joblib.load("models/current_classifier.joblib")
loaded_model_P = joblib.load("models/power_classifier.joblib")
loaded_model_Temp = joblib.load("models/temperature_classifier.joblib")
loaded_model_Vib = joblib.load("models/vibration_classifier.joblib")

predict status

In [4]:
df = pd.read_csv("dataset/preprocess.csv")
features_V = ["Vr", "Vy", "Vb", "voltage_RMS"]  # (voltage)
features_I = ["Ir", "Iy", "Ib", "current_RMS"]  # (current)
features_P = ["voltage_RMS", "current_RMS", "power"]  # (power)
features_Temp = ["MNDE_temp", "MDE_temp", "PNDE_temp", "PDE_temp"]  # (temperature)
features_Vib = ["MNDE_vib", "MDE_vib", "PNDE_vib", "PDE_vib"]  # (vibration)

df["V_class"] = loaded_model_V.predict(df[features_V])
df["I_class"] = loaded_model_I.predict(df[features_I])
df["P_class"] = loaded_model_P.predict(df[features_P])
df["temp_class"] = loaded_model_Temp.predict(df[features_Temp])
df["vib_class"] = loaded_model_Vib.predict(df[features_Vib])

c:\Users\arn48\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\arn48\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\arn48\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\arn48\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning:

In [5]:
data = [
    "pump_Id",
    "timestamp",
    "voltage_RMS",
    "current_RMS",
    "power",
    "MNDE_temp",
    "MDE_temp",
    "PNDE_temp",
    "PDE_temp",
    "MNDE_vib",
    "MDE_vib",
    "PNDE_vib",
    "PDE_vib",
    "V_class",
    "I_class",
    "P_class",
    "temp_class",
    "vib_class",
]

df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values(by="timestamp")
df[data].shape
df[data]

,pump_Id,timestamp,voltage_RMS,current_RMS,power,MNDE_temp,MDE_temp,PNDE_temp,PDE_temp,MNDE_vib,MDE_vib,PNDE_vib,PDE_vib,V_class,I_class,P_class,temp_class,vib_class
0,Pump 01,2024-02-26 15:34:35,6588.668811,128.541453,1279.688372,41.160795,41.631619,7.821142,46.288155,1.898533,1.997576,3.377428,3.046781,0,0,0,0,0
1,Pump 01,2024-02-26 16:21:55,6847.180891,123.746598,1284.280531,41.376192,41.881324,11.345882,46.366765,1.921614,1.955065,3.111335,3.118033,0,0,0,0,0
2,Pump 01,2024-02-26 16:24:39,6815.572916,123.836849,1280.255048,41.617898,41.896968,11.188060,46.253020,1.874649,1.940525,3.247729,3.254200,0,0,0,0,0
3,Pump 01,2024-02-26 16:27:28,6803.016663,123.581109,1276.039432,41.573761,41.981996,10.465278,46.277501,1.950923,1.953697,3.330099,2.974058,0,0,0,0,0
4,Pump 01,2024-02-26 16:33:02,6781.164679,125.414213,1288.901498,41.573042,41.940255,9.778056,46.294573,1.859869,1.932941,3.207304,3.168624,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,Pump 01,2024-02-29 10:24:44,6456.462341,130.592121,1269.461178,39.701498,39.210300,14.842343,45.836668,1.906820,1.966785,3.063787,3.154336,0,1,0,0,0
891,Pump 01,2024-02-29 10:41:24,6478.080201,130.007653,1269.083277,39.798443,39.416305,14.516340,45.827065,1.888813,1.951198,2.740483,3.050897,0,1,0,0,0
892,Pump 01,2024-02-29 10:46:54,6514.911554,129.901019,1275.908528,39.746720,39.544325,13.821767,45.811373,1.869356,2.005211,3.079876,3.036935,0,1,0,0,0
893,Pump 01,2024-02-29 10:52:29,6495.594415,130.107868,1272.385739,39.573242,39.574776,13.832138,45.840963,1.866290,1.957083,3.075060,2.520469,0,1,0,0,0


In [6]:
con = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=Nishanth\SQLEXPRESS;"
    "Database=Pump_data;"
    "Trusted_Connection=yes;"
)

cursor =con.cursor()

In [7]:
# Initialize an empty list to store rows for the log file
log_rows = []

for row in df[data].itertuples():

    cursor.execute(

        """INSERT INTO Pump_data.dbo.pump_status (pump_id, date_time, date_time_copy, voltage_RMS, current_RMS, power, MNDE_temp, MDE_temp, PNDE_temp, PDE_temp, MNDE_vib, MDE_vib, PNDE_vib, PDE_vib, V_class, I_class, P_class, temp_class, vib_class) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",

        row.pump_Id,
        row.timestamp,
        row.timestamp,
        row.voltage_RMS,
        row.current_RMS,
        row.power,
        row.MNDE_temp,
        row.MDE_temp,
        row.PNDE_temp,
        row.PDE_temp,
        row.MNDE_vib,
        row.MDE_vib,
        row.PNDE_vib,
        row.PDE_vib,
        row.V_class,
        row.I_class,
        row.P_class,
        row.temp_class,
        row.vib_class,

    )
    con.commit()
    time.sleep(1)
    
    # Save the row data to a log file
    if row.P_class in [1, 2]:
        # Append the row data to the log_rows list
        log_rows.append(row)

# Create a DataFrame from the log_rows list
log_df = pd.DataFrame(log_rows)

# Save the DataFrame to a CSV file
log_df.to_csv("log_file.csv",index=False)

KeyboardInterrupt: 